# USW-AI-Projekt: Einfluss sozialer Medien auf Aktienkurse
## CRISP-DM Struktur
---

### 1. Business Understanding
- Ziel: Zusammenhang zwischen Twitter-Stimmung und Aktienkursentwicklung untersuchen
- Zielgruppe: Finanzanalysten, Anleger
- Fragestellung: Kann man Kursbewegungen durch Stimmung erklären oder sogar vorhersagen?

### 2. Data Understanding
- Datenquellen: Twitter API v2, yfinance
- Zeitraum: z. B. letzte 3–6 Monate
- Unternehmen: z. B. SAP, Siemens, BMW

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats as stat

%matplotlib inline
sns.set()

# To avoid Warning message inbetween ...
import warnings
warnings.filterwarnings('ignore')

### 3. Data Preparation
Dataset loading

In [6]:
import pandas as pd

company_tweet = pd.read_csv("../Datasets/raw/tweets_top_companies/Company_Tweet.csv")
tweet         = pd.read_csv("../Datasets/raw/tweets_top_companies/Tweet.csv")
company       = pd.read_csv("../Datasets/raw/tweets_top_companies/Company.csv")
company_value = pd.read_csv("../Datasets/raw/company_values/CompanyValues.csv")


In [ ]:
# Platzhalter: Sentimentanalyse vorbereiten
# z. B. Huggingface Transformers laden und Tweets analysieren

### 4. Modelling
- Korrelation analysieren
- Optional: Klassifikation ↑/↓ mittels Logistic Regression

In [ ]:
# Beispiel: Pearson-Korrelation zwischen Stimmung & Kursveränderung berechnen

### 5. Evaluation
- Visualisierung mit matplotlib/seaborn/plotly
- Streudiagramm, Zeitreihenvergleich

In [ ]:
# Beispiel: Zeitreihe mit Kurs vs. Stimmung visualisieren

### 6. Ergebnis & Reporting
- Interpretation der Ergebnisse
- Empfehlung: Lohnt sich Social Media Analyse?
- ggf. Streamlit-Dashboard oder PDF-Bericht